## Compression Thoughts

TODO: Shuffle indexes once to simulate random access pattern then always index the same RGB bands and calculate the mean of each
to simulate _some_ computation measure the time it takes and then move the benchmark folder to a compressed folder!
I can check locally via BTRFS or squashfs or ...

But instinctively, I would assume that the block compression size should have to be aligned
with the size of the lmdb file 

-> Block compression is too inflexible and not configurable enough! (the blocks are too small to get good compressions)



In [ ]:
# test size
import lmdb
import blosc2
from numpy import random
from safetensors.numpy import save
from more_itertools import chunked

rand_band = lambda size: random.randint(2**15, size=(size, size)).astype("uint16")
rand_s2 = lambda: {
    "B01": rand_band(30),
    "B02": rand_band(120),
    "B03": rand_band(120),
    "B04": rand_band(120),
    "B05": rand_band(60),
    "B06": rand_band(60),
    "B07": rand_band(60),
    "B08": rand_band(120),
    "B8A": rand_band(60),
    "B09": rand_band(60),
    "B11": rand_band(60),
    "B12": rand_band(30),
}

In [ ]:
d = rand_s2()
a = d["B02"]

# len(save(rand_s2())) / 2**10
# a.nbytes / (4*2**10)
c = blosc2.compress2(
    a,
    typesize=a.dtype.itemsize,
    codec=blosc2.Codec.ZSTD,
    filters=[blosc2.Filter.BYTEDELTA],
)
len(c) / a.nbytes 

In [ ]:
sf_tns = save(sample_data)

c_d = blosc2.compress2(
    sf_tns,
    # typesize=a.dtype.itemsize,
    # codec=blosc2.Codec.ZSTD,
    codec=blosc2.Codec.ZSTD,
    clevel=9,
    # filters=[blosc2.Filter.BYTEDELTA],
    filters=[blosc2.Filter.BITSHUFFLE],
    # filters=[blosc2.Filter.SHUFFLE],
)

len(c_d) / len(sf_tns)

In [ ]:
# TODO: fix seed
env = lmdb.open("benchmark", map_size=2**40)
for chunk in chunked(range(10_000), 1_000):
    with env.begin(write=True) as txn:
        for i in chunk:
            txn.put(f"{i}".encode(), save(rand_s2()))